In [42]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
import pandas as pd
import os
from smolagents import OpenAIServerModel
from smolagents import CodeAgent, WebSearchTool, LiteLLMRouterModel
import requests

In [66]:
# Test df
df = pd.read_csv("100recipes.csv")

## Load Some Embedding model:

In [44]:
model_emb = SentenceTransformer("avsolatorio/GIST-large-Embedding-v0")

In [45]:
# Combine relevant text fields into one string per recipe
def make_full_text(row):
    ingredients = " ".join(eval(row["ingredients"])) if isinstance(row["ingredients"], str) else ""
    directions = " ".join(eval(row["directions"])) if isinstance(row["directions"], str) else ""
    return f"{row['title']} {ingredients} {directions}"

df["full_text"] = df.apply(make_full_text, axis=1)

## Embed the first 100 test recipies

In [46]:
texts = model_emb.encode(df.full_text, show_progress_bar= True)

Batches: 100%|██████████| 4/4 [00:12<00:00,  3.10s/it]


## Query Extpansion

In [47]:
question = input()

In [48]:
url = "http://localhost:1234/v1/chat/completions"
headers = {"Content-Type": "application/json"}

data = {
    "model": "qwen3-0.6b",
    "messages": [
        {"role": "system", "content": """You are a helpful assistant for query expansion. 
        You are given a user question but you shall not answer the users question. You are supposed to expand the question with keywords relevant to the question.
        Return keywords that fit specifically to the users input.
        For example: user: "I want to eat something italian." Your Answer:"Italian, food, Pizza, coffe, pasta, lasagna"  """},
        {"role": "user", "content": f"{question}"}
    ],
    "temperature": 0.1,
    "max_tokens": 256,
    "stream": False
}

response = requests.post(url, headers=headers, json=data)
print(response.json()["choices"][0]["message"]["content"])


<think>
Okay, the user wants to eat something German and has chicken and potatoes at home. Let me think about the keywords that fit here.

First, "German" is obvious. Then, they mentioned having chicken and potatoes. So maybe include those in there. Also, since it's food, adding "something" or "food" would make sense. Maybe "some" to indicate quantity? Wait, but the user says "I do have some", so "some" could be a keyword. Let me check again: Italian, food, pizza, coffee, pasta, lasagna. Oh right, that's from the example given. So in this case, German is the main theme, and the ingredients are chicken and potatoes. So the keywords would be German, chicken, potatoes, food. That should cover it.
</think>

Keywords: German, food, chicken, potatoes.


In [49]:
raw_query = response.json()["choices"][0]["message"]["content"]
_, q_ext = raw_query.split('</think')

In [50]:
q_ext + question

'>\n\nKeywords: German, food, chicken, potatoes.I want to eat somethin German. I do have some chicken and potatoes at home'

In [51]:
question_vec = model_emb.encode(question + q_ext)


In [60]:
import torch

In [62]:
similarities = model_emb.similarity(texts, question_vec)
similarities[0:10]

tensor([[0.3500],
        [0.4817],
        [0.3925],
        [0.5028],
        [0.3736],
        [0.5029],
        [0.3478],
        [0.3547],
        [0.3385],
        [0.3720]])

## Get the 3 best Recepies

In [65]:
top_k = torch.topk(similarities.squeeze(), k=3)
top_indices = top_k.indices
print(df.iloc[top_indices])

    Unnamed: 0.1  Unnamed: 0             title  \
40            40          40      Chicken Stew   
76            76          76  Consomme Chicken   
31            31          31    Summer Chicken   

                                          ingredients  \
40  ["3 lb. chicken, boiled", "4 medium potatoes, ...   
76  ["chicken parts", "1 can cream of chicken soup...   
31  ["1 pkg. chicken cutlets", "1/2 c. oil", "1/3 ...   

                                           directions  \
40  ["Remove chicken from bone.", "Use the broth."...   
76  ["Add enough liquid to chicken soup and consom...   
31                ["Double recipe for more chicken."]   

                                                link    source  \
40   www.cookbooks.com/Recipe-Details.aspx?id=284237  Gathered   
76  www.cookbooks.com/Recipe-Details.aspx?id=1078857  Gathered   
31   www.cookbooks.com/Recipe-Details.aspx?id=969444  Gathered   

                                                  NER  \
40  ["chicken", "po

for later agentic usage:

In [ ]:
model = OpenAIServerModel(
    model_id="qwen3-0.6b",
    api_base="http://localhost:1234/v1",
    api_key= "not-needed",
)

In [ ]:
# agent = CodeAgent(tools=[WebSearchTool()], model=model)
# agent.run("How many seconds would it take for a leopard at full speed to run through Pont des Arts?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How many seconds would it take for a leopard at full speed to run through Pont des Arts?                        │
│                                                                                                                 │
╰─ OpenAIServerModel - qwen3-0.6b ────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  print("It would take approximately 2 to 3 minutes for a leopard at full speed to run through Pont des Arts.")    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
It would take approximately 2 to 3 minutes for a leopard at full speed to run through Pont des Arts.

Out: None

[Step 1: Duration 5.30 seconds| Input tokens: 2,059 | Output tokens: 521]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 10 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("120 seconds")                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 120 seconds

[Step 10: Duration 1.34 seconds| Input tokens: 29,010 | Output tokens: 2,859]

'120 seconds'